Created an account in Stability AI and got the API key.
Added that in the OS environment variable. 
Listed out the available models, to choose one from.

In [38]:
import os
import requests
import json
api_key=os.getenv("STABILITY_API_KEY")
api_host = "https://api.stability.ai"
url = f"{api_host}/v1/engines/list"
response = requests.get(url, headers={"Authorization": f"Bearer {api_key}"})
payload = response.json()

payload = json.dumps(payload, indent=2)
print ("Following are the list of available models \n\n")
print(payload)

Following are the list of available models 


[
  {
    "description": "Stability-AI Stable Diffusion XL v1.0",
    "id": "stable-diffusion-xl-1024-v1-0",
    "name": "Stable Diffusion XL v1.0",
    "type": "PICTURE"
  },
  {
    "description": "Stability-AI Stable Diffusion v1.6",
    "id": "stable-diffusion-v1-6",
    "name": "Stable Diffusion v1.6",
    "type": "PICTURE"
  }
]


I am using stable-diffusion-xl-1024-v1-0 engine ID.
cfg_scale is set to 30 to make sure that the generated image is closer to the prompt.
samples is set to one, so that only one image is getting generated.
steps option is set to 49, so 49 diffusion steps are added making it a clearer image.  When I made this more than 50, it errored out.



In [41]:
import base64
import os
import requests
import datetime
import re

api_key = os.getenv("STABILITY_API_KEY")
prompt = "Astronaut washing utensils in space"
# Set the folder to save the image; make sure it exists
curdir = r"C:\Users\Riaz\Desktop\MSDS\Generative AI\Week 4"
image_dir = os.path.join(curdir, 'images')
engine_id = "stable-diffusion-xl-1024-v1-0"
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)
# Function to clean up filenames
def valid_filename(s):
    s = re.sub(r'[^\w_.)( -]', '', s).strip()
    return re.sub(r'[\s]+', '_', s)
response = requests.post(f"{api_host}/v1/generation/{engine_id}/text-to-image",headers={"Content-Type": "application/json",
                                                                                        "Accept": "application/json","Authorization": f"Bearer {api_key}"},json={"text_prompts": [{"text": f"{prompt}",}],"cfg_scale": 30,"height": 1024,"width": 1024,"samples": 1,"steps": 49,},)
data = response.json()
for i, image in enumerate(data["artifacts"]):
    filename = f"sd_panda.png"
    image_path = os.path.join(image_dir, filename)
    with open(image_path, "wb") as f:
        f.write(base64.b64decode(image["base64"]))

The original image which needs editing is being passed and the input prompt is suitably modified to edit the photo and produce the output.
DDIM sampler is being tried out here to give faster results.

In [43]:
orginal_image=r'C:\Users\Riaz\Desktop\MSDS\Generative AI\Week 4\images\sd_panda.png'
response = requests.post(f"{api_host}/v1/generation/{engine_id}/image-to-image",
                         headers={"Accept": "application/json","Authorization": f"Bearer {api_key}"},
                         files={"init_image": open(orginal_image, "rb")},
                         data={"image_strength": 0.35,"init_image_mode": "IMAGE_STRENGTH","text_prompts[0][text]":
                               "without water","cfg_scale": 7,"samples": 1,"steps": 50,"sampler": "DDIM"})
data = response.json()
for i, image in enumerate(data["artifacts"]):
    filename = f"{valid_filename(os.path.basename(orginal_image))}_img2img_{i}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    image_path = os.path.join(image_dir, filename)
with open(image_path, "wb") as f:
    f.write(base64.b64decode(image["base64"]))

Analysis:
    
    It is quite simple to use an Stable diffusion API and create the image.  It is also easy to edit the image, as long as we are uploading the right image and our desired prompting.
    The image created for "Astronaut washing utensils in space" created a image closer to the prompt.
    In the next prompt for editing it using the image-to-image API, "without water" prompt has been used.  This created an almost similar image, but few water drops missing on the utensils. Overall a good editing.